<h1>Segmentation and Clustering of Neighbourhoods in Toronto </h1>

Importing the required python libraries

In [1]:
import requests
import pandas as pd
from bs4 import BeautifulSoup

<h3>Data Collection</h3>

Extracting Table Data from the Wikipedia Page using Beautiful Soup

In [2]:
website_url = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text

soup = BeautifulSoup(website_url, 'lxml')
my_table = soup.find('table', {'class':'wikitable sortable'})
my_table

<table class="wikitable sortable">
<tbody><tr>
<th>Postcode</th>
<th>Borough</th>
<th>Neighbourhood
</th></tr>
<tr>
<td>M1A</td>
<td>Not assigned</td>
<td>Not assigned
</td></tr>
<tr>
<td>M2A</td>
<td>Not assigned</td>
<td>Not assigned
</td></tr>
<tr>
<td>M3A</td>
<td><a href="/wiki/North_York" title="North York">North York</a></td>
<td><a href="/wiki/Parkwoods" title="Parkwoods">Parkwoods</a>
</td></tr>
<tr>
<td>M4A</td>
<td><a href="/wiki/North_York" title="North York">North York</a></td>
<td><a href="/wiki/Victoria_Village" title="Victoria Village">Victoria Village</a>
</td></tr>
<tr>
<td>M5A</td>
<td><a href="/wiki/Downtown_Toronto" title="Downtown Toronto">Downtown Toronto</a></td>
<td><a href="/wiki/Harbourfront_(Toronto)" title="Harbourfront (Toronto)">Harbourfront</a>
</td></tr>
<tr>
<td>M5A</td>
<td><a href="/wiki/Downtown_Toronto" title="Downtown Toronto">Downtown Toronto</a></td>
<td><a href="/wiki/Regent_Park" title="Regent Park">Regent Park</a>
</td></tr>
<tr>
<td>M6A</td>

<h4>Parsing Table Data and storing in a Pandas Dataframe</h4>

Getting Table Header for Column Names

In [3]:
th = my_table.findAll('th')

header = []

for t in th:
    header.append(t.get_text())
    
header[len(header)-1] = header[len(header)-1][:len(header[len(header)-1])-1] #done to remove the escape charachters of any last value in series like '\n'
header

['Postcode', 'Borough', 'Neighbourhood']

Getting all Data Column-Wise using 'Slicing of Tuples'

In [4]:
td = my_table.findAll('td')

_postcode = []
_borough = []
_neighbourhood = []

td_Postcode = td[::3]
for t in td_Postcode:
    _postcode.append(t.get_text())

td_Borough = td[1:][::3]
for t in td_Borough:
    _borough.append(t.get_text())

td_Neighbourhood = td[2:][::3]
for t in td_Neighbourhood:
    _neighbourhood.append(t.get_text()[:(len(t.get_text())-1)])

Compiling all data and storing it in a Pandas Dataframe

In [5]:
df = pd.DataFrame(columns = header)

df['Postcode'] = _postcode
df['Borough'] = _borough
df['Neighbourhood'] = _neighbourhood

df.head(10)

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights
7,M6A,North York,Lawrence Manor
8,M7A,Queen's Park,Not assigned
9,M8A,Not assigned,Not assigned


<h3>Data Wrangling</h3>

Dropping rows where Borough is set to 'Not assigned'

In [6]:
df = df[df.Borough != 'Not assigned']
df.reset_index(inplace=True)

Grouping Neighbourhoods with the same Postal Code

In [7]:
df['Neighbourhood'].replace('Not assigned', df.Borough, inplace=True)
df = df.groupby('Postcode').agg({'Borough':'first', 'Neighbourhood':', '.join}).reset_index()
df

,Postcode,Neighbourhood,Borough
0,M1B,"Rouge, Malvern",Scarborough
1,M1C,"Highland Creek, Rouge Hill, Port Union",Scarborough
2,M1E,"Guildwood, Morningside, West Hill",Scarborough
3,M1G,Woburn,Scarborough
4,M1H,Cedarbrae,Scarborough
5,M1J,Scarborough Village,Scarborough
6,M1K,"East Birchmount Park, Ionview, Kennedy Park",Scarborough
7,M1L,"Clairlea, Golden Mile, Oakridge",Scarborough
8,M1M,"Cliffcrest, Cliffside, Scarborough Village West",Scarborough
9,M1N,"Birch Cliff, Cliffside West",Scarborough


Describing Data

In [8]:
df.shape

(103, 3)